In [ ]:
# --- Step 1: ライブラリの読み込みとパス定義 ---

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# パス定義
DATA_DIR = Path('../../data/circle-text')
CSV_PATH = DATA_DIR / 'metadata.csv'
IMG_DIR = DATA_DIR / 'images'

# デバイス設定（CUDA > MPS > CPU）
device = torch.device('cuda' if torch.cuda.is_available() else
                      'mps' if torch.backends.mps.is_available() else 'cpu')

print("Using device:", device)

In [ ]:
# --- Step 2: 動的ペア生成型Datasetの実装 ---

import random

class DynamicPairRankingDataset(Dataset):
    """
    ランキング付き画像リストから、ランダムな画像ペアとラベル（-1, 0, 1）を返すDataset
    """
    def __init__(self, csv_path, img_dir, transform=None, indices=None):
        self.df = pd.read_csv(csv_path)
        if indices is not None:
            self.df = self.df.iloc[indices].reset_index(drop=True)
        self.img_dir = Path(img_dir)
        self.transform = transform

    def __len__(self):
        # 1エポックあたりのペア数（画像数と同じくらいにしておく）
        return len(self.df)

    def __getitem__(self, idx):
        # 2つの異なる画像をランダムに選ぶ
        idx1, idx2 = random.sample(range(len(self.df)), 2)
        row1 = self.df.iloc[idx1]
        row2 = self.df.iloc[idx2]

        img1_path = self.img_dir / f"img_{int(row1['id'])}.png"
        img2_path = self.img_dir / f"img_{int(row2['id'])}.png"
        img1 = Image.open(img1_path).convert('RGB')
        img2 = Image.open(img2_path).convert('RGB')
        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        # ラベル付与
        if row1['score'] > row2['score']:
            label = 0  # 左（img1）が良い
        elif row1['score'] < row2['score']:
            label = 1  # 右（img2）が良い
        else:
            label = -1  # 同等

        return img1, img2, label

In [ ]:
# --- Step 3: モデル定義だよ！ギャルもCNNでバチバチに判定しちゃうからヨロシク！---

class PairwiseRankingNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 画像1枚ずつ特徴抽出するよ
        self.feature = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 32 * 32, 128), nn.ReLU()
        )
        # 2枚分の特徴をconcatして3クラス分類するよ
        self.classifier = nn.Sequential(
            nn.Linear(128 * 2, 64), nn.ReLU(),
            nn.Linear(64, 3)  # 3クラス（左が良い・同じ・右が良い）
        )

    def forward(self, img1, img2):
        feat1 = self.feature(img1)
        feat2 = self.feature(img2)
        x = torch.cat([feat1, feat2], dim=1)
        out = self.classifier(x)
        return out  # (B, 3) softmaxはlossでやるからナシ！

# ギャルもtransformは大事にするからね！
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [ ]:
# --- Step 4: 損失関数と最適化だよ！ギャルはクロスエントロピーでバチバチに決めるから！---

# モデルをデバイスに乗せるよ
model = PairwiseRankingNet().to(device)

# ギャルはクロスエントロピーで3クラス分類！
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
# --- Step 5: データ分割とDataLoaderだよ！ギャルは検証もバッチリやるから安心して！---

# データをtrain/valに分けるよ
df = pd.read_csv(CSV_PATH)
train_idx, val_idx = train_test_split(df.index, test_size=0.2, random_state=42)

# ギャルのペアデータセットを作るよ
train_dataset = DynamicPairRankingDataset(CSV_PATH, IMG_DIR, transform=transform, indices=train_idx)
val_dataset = DynamicPairRankingDataset(CSV_PATH, IMG_DIR, transform=transform, indices=val_idx)

# DataLoaderでバッチもバッチリ！
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=